# Simulation in **hypothesis testing** and **Confidence Interval**(CI) with **invalid IVs** for the 2SLS, PT-2SLS, and the proposed 2SIR methods.

In [1]:
from nl_causal.ts_models import _2SLS, _2SIR
from nl_causal.linear_reg import L0_IC
import numpy as np
from sklearn.preprocessing import normalize
from sim_data import sim
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import power_transform, quantile_transform

In [2]:
## simulate a dataset
np.random.seed(0)
n, p = 5000, 50
beta0 = 0.0
theta0 = np.random.randn(p)
theta0 = theta0 / np.sqrt(np.sum(theta0**2))
alpha0 = np.zeros(p)
alpha0[:5] = 1.
Z, X, y, phi = sim(n, p, theta0, beta0, alpha0=alpha0, case='inverse', feat='AP-normal')
## normalize the dataset
center = StandardScaler(with_std=False)
mean_X, mean_y = X.mean(), y.mean()
Z, X, y = center.fit_transform(Z), X - mean_X, y - mean_y
y_scale = y.std()
y = y / y_scale
## generate two-stage dataset
Z1, Z2, X1, X2, y1, y2 = train_test_split(Z, X, y, test_size=0.5, random_state=42)
n1, n2 = len(Z1), len(Z2)
LD_Z1, cov_ZX1 = np.dot(Z1.T, Z1), np.dot(Z1.T, X1)
LD_Z2, cov_ZY2 = np.dot(Z2.T, Z2), np.dot(Z2.T, y2)

In [8]:
## 2SLS
# specify a sparse regression model to detect invalid IVs
Ks = range(p)
reg_model = L0_IC(fit_intercept=False, alphas=10**np.arange(-1,3,.3),
				Ks=Ks, max_iter=10000, refit=False, find_best=False)
LS = _2SLS(sparse_reg=reg_model)
## Stage-1 fit theta 
LS.fit_theta(LD_Z1, cov_ZX1)
## Stage-2 fit beta
LS.fit_beta(LD_Z2, cov_ZY2, n2)
## produce p_value for beta
LS.test_effect(n2, LD_Z2, cov_ZY2)
print('p-value for 2SLS: %.5f' %LS.p_value)

LS.CI_beta(n1, n2, Z1, X1, LD_Z2, cov_ZY2, level=0.95)
LS.CI[0] = max(LS.CI[0], 0.)
print('CI for 2SLS: %s' %(LS.CI*y_scale))

LS.selection_summary()

p-value for 2SLS: 0.03440
CI for 2SLS: [0.         0.46709488]


,candidate_model,criteria,mse
0,"(0, 1, 2, 3, 4, 32, 35, 41, 46, 50)",1.043018,0.178870
1,"(0, 1, 2, 3, 4, 6, 7, 9, 11, 15, 16, 19, 22, 2...",1.092367,0.177081
2,"(0, 1, 2, 3, 4, 36, 6, 7, 41, 44, 46, 15, 22, ...",1.060029,0.178557
3,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14,...",1.147092,0.176797
4,"(0, 1, 2, 3, 4, 36, 6, 7, 37, 41, 32, 44, 46, ...",1.057327,0.178080
...,...,...,...
100,"(0, 1, 2, 3, 4, 36, 6, 7, 37, 41, 44, 45, 46, ...",1.066623,0.178063
101,"(0, 1, 3, 50)",1.547436,0.271369
102,"(0, 1, 2, 3, 4, 36, 44, 46, 50)",1.042585,0.179346
103,"(0, 1, 2, 3, 4, 6, 7, 8, 9, 11, 13, 14, 15, 16...",1.113104,0.176874


In [7]:
## PT-2SLS
Ks = range(p)
reg_model = L0_IC(fit_intercept=False, alphas=10**np.arange(-1,3,.3),
				Ks=Ks, max_iter=10000, refit=False, find_best=False)
PT_X1 = power_transform(X1.reshape(-1,1), method='yeo-johnson').flatten()
PT_cor_ZX1 = np.dot(Z1.T, PT_X1)
PT_LS = _2SLS(sparse_reg=reg_model)
## Stage-1 fit theta
PT_LS.fit_theta(LD_Z1, PT_cor_ZX1)
## Stage-2 fit beta
PT_LS.fit_beta(LD_Z2, cov_ZY2, n2)
## produce p-value for beta
PT_LS.test_effect(n2, LD_Z2, cov_ZY2)
print('p-value for PT-2SLS: %.5f' %PT_LS.p_value)

PT_LS.CI_beta(n1, n2, Z1, X1, LD_Z2, cov_ZY2, level=0.95)
PT_LS.CI[0] = max(PT_LS.CI[0], 0.)
print('CI for PT-2SLS: %s' %(PT_LS.CI*y_scale))

LS.selection_summary()

p-value for PT-2SLS: 0.84130
CI for PT-2SLS: [0.         0.07259953]


,candidate_model,criteria,mse
0,"(0, 1, 2, 3, 4, 32, 35, 41, 46, 50)",1.043018,0.178870
1,"(0, 1, 2, 3, 4, 6, 7, 9, 11, 15, 16, 19, 22, 2...",1.092367,0.177081
2,"(0, 1, 2, 3, 4, 36, 6, 7, 41, 44, 46, 15, 22, ...",1.060029,0.178557
3,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14,...",1.147092,0.176797
4,"(0, 1, 2, 3, 4, 36, 6, 7, 37, 41, 32, 44, 46, ...",1.057327,0.178080
...,...,...,...
100,"(0, 1, 2, 3, 4, 36, 6, 7, 37, 41, 44, 45, 46, ...",1.066623,0.178063
101,"(0, 1, 3, 50)",1.547436,0.271369
102,"(0, 1, 2, 3, 4, 36, 44, 46, 50)",1.042585,0.179346
103,"(0, 1, 2, 3, 4, 6, 7, 8, 9, 11, 13, 14, 15, 16...",1.113104,0.176874


In [6]:
## the proposed 2SIR
Ks = range(p)
reg_model = L0_IC(fit_intercept=False, alphas=10**np.arange(-1,3,.3),
				Ks=Ks, max_iter=10000, refit=False, find_best=False)
SIR = _2SIR(sparse_reg=reg_model)
## Stage-1 fit theta
SIR.fit_theta(Z1, X1)
## Stage-2 fit beta
SIR.fit_beta(LD_Z2, cov_ZY2, n2)
## generate CI for beta
SIR.test_effect(n2, LD_Z2, cov_ZY2)
print('p-value for 2SIR: %.5f' %SIR.p_value)

SIR.CI_beta(n1, n2, Z1, X1, LD_Z2, cov_ZY2, level=0.95)
print('CI for 2SLS: %s' %(SIR.CI*y_scale))

SIR.selection_summary()

p-value for 2SIR: 0.81863
CI for 2SLS: [0.         0.06909517]


,candidate_model,criteria,mse
0,"(0, 1, 2, 3, 4, 36, 35, 7, 37, 41, 5, 44, 46, ...",1.066233,0.177994
1,"(0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 16, 22, 23, 24...",1.084349,0.177324
2,"(0, 1, 2, 3, 4, 5, 6, 7, 11, 16, 22, 23, 24, 3...",1.079011,0.177487
3,"(0, 1, 2, 3, 4, 36, 5, 35, 7, 41, 32, 44, 46, ...",1.058361,0.178262
4,"(0, 1, 2, 3, 4, 5, 7, 9, 16, 22, 23, 26, 31, 3...",1.083622,0.177195
...,...,...,...
108,"(0, 1, 2, 3, 4, 46, 50)",1.039468,0.179902
109,"(2, 3, 50)",2.338659,0.411808
110,"(0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 16, 22, 23, 24...",1.081364,0.177349
111,"(0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 16, 21, 22...",1.098011,0.176973
